### **Cell 1: Setup, Geometry, and Vectorized Definitions**

**Mathematical Context:**
This cell establishes the algebraic geometry framework for a hyperelliptic curve of genus $g=30$.

1.  **The Curve:** We define a hyperelliptic curve $\mathcal{C}$ given by the equation:
    $$y^2 = \prod_{i=1}^{2g+2} (x - a_i)$$
    where $\{a_i\}$ are the $2g+2 = 62$ branch points. These points are generated randomly on a perturbed circle in the complex plane to ensure general position.

2.  **Holomorphic Differentials:** The basis for the space of holomorphic differentials $H^0(\mathcal{C}, \Omega^1)$ is defined as:
    $$\omega_k = \frac{z^k}{\sqrt{\prod_{i} (z - a_i)}} \, dz, \quad \text{for } k = 0, \dots, g-1$$

3.  **MPS Optimization (Log-Sum-Exp):**
    For $g=30$, the denominator involves a polynomial of degree 62. Evaluating this directly causes floating-point overflow. To solve this on the GPU (MPS), we compute the differential in log-space:
    $$ \log(\omega_k) = k \log(z) - \frac{1}{2} \sum_{i} \log(z - a_i) $$
    $$ \omega_k = \exp(\log(\omega_k)) $$
    This ensures numerical stability using `torch.complex64` or `torch.complex128`.

4.  **Vectorized Integration:**
    We define a custom Riemann-sum integrator `integrate_row_mps` that computes the path integral $\int_{P_0}^{z} \omega_k$ for an entire row of target points $z$ simultaneously, replacing scalar CPU integration methods like `mp.quad`.

---


In [43]:
# @title Setup: install, imports, mount Drive, shared config, genus-30 cuts (MPS Version)
!pip install -q torch numpy mpmath tqdm

import os, time, math, json, numpy as np, torch, mpmath as mp
from tqdm import tqdm

# -----------------------------
# 1. Device & Precision Setup
# -----------------------------
# Mac MPS usually prefers Float32 for speed, but we try Float64/Complex128 for stability if available.
# If you get "MPS not implemented" errors, switch `dtype` to torch.complex64
# DEVICE = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
elif torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
else:
    DEVICE = torch.device("cpu")

print(f"Using device: {device}")

DTYPE = torch.complex64 # Genus 30 is heavy; Complex64 is safer for MPS speed/compatibility but can switch to complex128
print(f"Running on Device: {DEVICE} | Precision: {DTYPE}")

# -----------------------------
# Drive & save directory
# -----------------------------

DRIVE_FOLDER = "AJ_Tables_g30"
# DRIVE_FOLDER = "AJ_Tables_g100"


SAVE_DIR = f"./{DRIVE_FOLDER}"
os.makedirs(SAVE_DIR, exist_ok=True)

OMEGAS_PATH    = os.path.join(SAVE_DIR, "aj_omegas_genus30.pt")
INTEGRALS_PATH = os.path.join(SAVE_DIR, "aj_integrals_genus30.pt")


# OMEGAS_PATH    = os.path.join(SAVE_DIR, "aj_omegas_genus100.pt")
# INTEGRALS_PATH = os.path.join(SAVE_DIR, "aj_integrals_genus100.pt")


print("Will save to:\n ", OMEGAS_PATH, "\n ", INTEGRALS_PATH)

# -----------------------------
# Genus & grid
# -----------------------------
genus = 30
# genus = 150
H = W = 96
r_min, r_max = -6.0, 6.0
i_min, i_max = -6.0, 6.0
grid_r = np.linspace(r_min, r_max, W).astype(np.float64)
grid_i = np.linspace(i_min, i_max, H).astype(np.float64)

# We keep mp.dps for the initial setup of branch cuts, but integration will be standard float
mp.mp.dps = 50
base_point_complex = complex(r_min - 2.0, i_min - 2.0) # Python complex type

# -----------------------------
# Helper: atomic save
# -----------------------------
def atomic_torch_save(obj, path):
    tmp = path + ".tmp"
    torch.save(obj, tmp)
    os.replace(tmp, path)

# -----------------------------
# Genus-30 cuts: (CPU Logic - runs once, keep as is)
# -----------------------------
def make_hyperelliptic_cuts(g, radius=4.0, jitter=0.25, seed=123):
    rng = np.random.RandomState(seed)
    m = 2*g + 2
    thetas = np.linspace(0, 2*np.pi, m, endpoint=False)
    rng.shuffle(thetas)
    radii = radius * (1.0 + jitter * (rng.rand(m) - 0.5))
    pts = radii * np.exp(1j * thetas)
    
    scale = max((pts.real.max()-pts.real.min())/(r_max-r_min+1e-6),
                (pts.imag.max()-pts.imag.min())/(i_max-i_min+1e-6))
    if scale > 0.85:
        pts = pts / (scale/0.85)

    remaining = list(range(m))
    cuts = []
    while remaining:
        i = remaining.pop(0)
        pi = pts[i]
        dists = [(j, abs(pi - pts[j])) for j in remaining]
        j = min(dists, key=lambda t: t[1])[0]
        remaining.remove(j)
        a, b = pi, pts[j]
        mid = 0.5*(a+b)
        a = a + 0.05*(a - mid)
        b = b + 0.05*(b - mid)
        cuts.append((complex(a), complex(b)))
    
    if len(cuts) > g+1:
        cuts.sort(key=lambda ab: -abs(ab[0]-ab[1]))
        cuts = cuts[:g+1]
    return cuts

branch_cuts = make_hyperelliptic_cuts(genus)
branch_pts_list = [a for ab in branch_cuts for a in ab]
print(f"genus={genus} → cuts={len(branch_cuts)} ; total branch points={len(branch_pts_list)}")

# -----------------------------
# PREPARE DATA FOR MPS
# -----------------------------
# Move branch points to GPU once.
# shape: (1, 1, num_branch_pts) to allow broadcasting against (Steps, Width, 1)
branch_pts_tensor = torch.tensor(branch_pts_list, device=DEVICE, dtype=DTYPE).reshape(1, 1, -1)

# -----------------------------
# VECTORIZED MATH (PyTorch/MPS)
# -----------------------------
def omega_torch(k, t_tensor):
    """
    Vectorized calculation of ω_k(t) = t^k / sqrt(prod(t - a_i))
    Uses Log-Sum-Exp trick to prevent overflow for Genus 30.
    """
    # t_tensor shape: (Steps, Width)
    # branch_pts_tensor shape: (1, 1, 62)
    
    # 1. Compute differences: (Steps, Width, 62)
    diffs = t_tensor.unsqueeze(-1) - branch_pts_tensor
    
    # 2. Compute Log of denominator: 0.5 * sum(log(diffs))
    # We use complex log. torch.log handles complex inputs correctly on recent versions.
    log_diffs = torch.log(diffs) 
    log_denom = 0.5 * torch.sum(log_diffs, dim=-1) # Sum over branch points -> (Steps, Width)
    
    # 3. Compute Log of numerator: k * log(t)
    # (Using log space for numerator too ensures we don't blow up t^30)
    log_num = k * torch.log(t_tensor)
    
    # 4. Combine and exponentiate
    # ω = exp(log_num - log_denom)
    log_omega = log_num - log_denom
    return torch.exp(log_omega)

def integrate_row_mps(k, z_row_tensor, base_pt, steps=400):
    """
    Computes integral from base_pt to every point in z_row_tensor simultaneously.
    Uses Trapezoidal/Riemann summation.
    
    z_row_tensor: (W,) complex tensor on MPS
    base_pt: scalar complex (python or tensor)
    steps: integration resolution (higher = slower but more accurate)
    """
    W_dim = z_row_tensor.size(0)
    
    # 1. Create Time Steps (0 to 1)
    # shape: (steps, 1)
    t = torch.linspace(0, 1, steps, device=DEVICE, dtype=z_row_tensor.dtype.real_dtype).unsqueeze(1)
    
    # 2. Broadcast Path
    # Path goes from base_pt to z_row[i]
    # diff shape: (W,)
    diff = z_row_tensor - base_pt
    
    # path shape: (steps, W)
    # base_pt + t * diff
    path = base_pt + (t.type(DTYPE) * diff.unsqueeze(0))
    
    # 3. Evaluate Derivative (dt)
    # dt = diff / steps (constant for straight line)
    # shape: (W,)
    dt = diff / steps
    
    # 4. Evaluate Omega on the whole grid
    # shape: (steps, W)
    vals = omega_torch(k, path)
    
    # 5. Sum (Trapezoidal approximation: simple sum * dt)
    # Result shape: (W,)
    # Summing along dim 0 (time steps)
    integral = torch.sum(vals, dim=0) * dt
    
    return integral

# -----------------------------
# Metadata
# -----------------------------
COMMON_META = {
    "genus": genus,
    "branch_cuts": branch_cuts,
    # Store simple numpy/list version in meta, not the tensor
    "branch_pts": np.array(branch_pts_list, dtype=np.complex128),
    "grid_r": grid_r, "grid_i": grid_i,
    "meta": {"dtype": str(DTYPE), "base_point": base_point_complex,
             "grid_shape": (H, W), "ranges": (r_min, r_max, i_min, i_max)}
}

def ensure_config_compatible(pkg):
    assert int(pkg["genus"]) == genus
    assert pkg["meta"]["grid_shape"] == (H, W)
    # Loosen strict equality for floats
    assert np.allclose(pkg["grid_r"], grid_r)

Using device: mps
Running on Device: mps | Precision: torch.complex128
Will save to:
  ./AJ_Tables_g30/aj_omegas_genus30.pt 
  ./AJ_Tables_g30/aj_integrals_genus30.pt
genus=30 → cuts=31 ; total branch points=62



### **Cell 2: Consistency Verification**

**Functional Description:**
This cell ensures data integrity when resuming a long computation.

1.  **Geometry Check:** It compares the currently generated branch points $\{a_i\}_{current}$ with the branch points stored in `aj_integrals_genus30.pt` (if it exists).
2.  **Metric:** It checks the Euclidean distance in the complex plane:
    $$ \max_i | a_i^{\text{saved}} - a_i^{\text{current}} | < \epsilon $$
3.  **Synchronization:** If a mismatch is found (which would render the integrals invalid), the code can optionally `AUTO_ADOPT` the saved geometry, overwriting the in-memory tensors to match the disk.

___

In [39]:
import os
import datetime
import torch
import numpy as np

# 1. File Modification Check
if 'INTEGRALS_PATH' in globals() and os.path.exists(INTEGRALS_PATH):
    mtime = os.path.getmtime(INTEGRALS_PATH)
    print(f"Last modification time of {os.path.basename(INTEGRALS_PATH)}: {datetime.datetime.fromtimestamp(mtime)}")
else:
    print(f"File not found or INTEGRALS_PATH not defined: {INTEGRALS_PATH if 'INTEGRALS_PATH' in globals() else 'Not Defined'}")

# @title Check/lock branch cuts against saved files (MPS Update)
# Toggle: if mismatch is found, automatically replace in-memory cuts/points with the saved ones.
AUTO_ADOPT_SAVED_CUTS = False   # set True to auto-fix

# Helper: pick a saved file to compare against
def pick_saved_path():
    cand = []
    if 'INTEGRALS_PATH' in globals() and os.path.exists(INTEGRALS_PATH):
        cand.append(INTEGRALS_PATH)
    if 'OMEGAS_PATH' in globals() and os.path.exists(OMEGAS_PATH):
        cand.append(OMEGAS_PATH)
    if not cand:
        return None
    cand.sort(key=lambda p: os.path.getmtime(p), reverse=True)
    return cand[0]

def np_c128_from_list(lst):
    return np.array([complex(z) for z in lst], dtype=np.complex128)

# Sanity check for Cell 0/1 variables
# Note: We use 'branch_pts_list' from the MPS Cell 1, but we alias it if needed
if 'branch_pts_list' in globals():
    current_pts_source = branch_pts_list
elif 'branch_pts' in globals():
    current_pts_source = branch_pts
else:
    raise RuntimeError("Expected `branch_pts_list` or `branch_pts` from Cell 1.")

saved_path = pick_saved_path()

if saved_path is None:
    print("No saved ω/Ι file found in Drive — looks like a fresh run; nothing to compare.")
else:
    # Load metadata on CPU
    pkg = torch.load(saved_path, map_location="cpu", weights_only=False)
    
    # Extract saved config
    saved_genus = int(pkg.get("genus"))
    saved_grid_r = np.array(pkg.get("grid_r"))
    saved_grid_i = np.array(pkg.get("grid_i"))
    # Handle list vs array in saved files
    saved_raw_pts = pkg.get("branch_pts", [])
    if isinstance(saved_raw_pts, torch.Tensor):
        saved_raw_pts = saved_raw_pts.numpy()
    saved_pts = np.array(saved_raw_pts, dtype=np.complex128)
    saved_cuts = pkg.get("branch_cuts", None)

    curr_pts = np_c128_from_list(current_pts_source)

    print(f"Comparing current session to: {os.path.basename(saved_path)}")
    ok_genus = (saved_genus == genus)
    ok_grid  = np.allclose(saved_grid_r, grid_r) and np.allclose(saved_grid_i, grid_i)
    
    # Robust length check (flatten if needed)
    ok_len = (saved_pts.size == curr_pts.size)
    
    # Robust value check
    if ok_len:
        saved_flat = saved_pts.flatten()
        curr_flat = curr_pts.flatten()
        ok_pts = np.allclose(saved_flat.real, curr_flat.real) and \
                 np.allclose(saved_flat.imag, curr_flat.imag)
        max_dev = float(np.max(np.abs(saved_flat - curr_flat))) if saved_flat.size else 0.0
    else:
        ok_pts = False
        max_dev = 999.9

    print(f"  genus match : {ok_genus} (saved={saved_genus}, current={genus})")
    print(f"  grid match  : {ok_grid}")
    if ok_len:
        print(f"  branch pts  : {'MATCH' if ok_pts else 'MISMATCH'} (max |Δ| = {max_dev:.3e})")
    else:
        print(f"  branch pts  : count differs (saved {saved_pts.size}, current {curr_pts.size})")

    if ok_genus and ok_grid and ok_pts:
        print("\n✅ SAFE to resume: cuts/points match what’s in Drive.")
    else:
        print("\n❌ MISMATCH detected — do NOT resume building tables with different cuts.")
        
        if AUTO_ADOPT_SAVED_CUTS and (saved_cuts is not None):
            # 1. Update CPU lists
            branch_cuts = saved_cuts
            branch_pts_list = [a for ab in branch_cuts for a in ab]
            
            # 2. IMPORTANT: Update the MPS Tensor used by the integrator
            # (Re-run the tensor creation line from Cell 1)
            if 'DEVICE' in globals() and 'DTYPE' in globals():
                branch_pts_tensor = torch.tensor(branch_pts_list, device=DEVICE, dtype=DTYPE).reshape(1, 1, -1)
                print("→ Updated `branch_pts_tensor` on MPS device.")
            
            print("→ Adopted the saved branch cuts/points into memory. You can now safely resume.")
        else:
            print("Tip: set AUTO_ADOPT_SAVED_CUTS=True at the top of this cell to adopt the saved cuts.")

File not found or INTEGRALS_PATH not defined: ./AJ_Tables_g30/aj_integrals_genus30.pt
No saved ω/Ι file found in Drive — looks like a fresh run; nothing to compare.




### **Cell 3: Evaluation of Differentials ($\omega$)**

**Mathematical Context:**
This cell computes the raw values of the differential forms on a discrete 2D grid.

1.  **The Grid:** We define a lattice $\Lambda$ of size $H \times W$ in the complex plane $\mathbb{C}$.
2.  **Computation:** For every genus index $k \in \{0, \dots, 29\}$ and every spatial point $z_{xy} \in \Lambda$, we compute:
    $$ V_{k, y, x} = \frac{z_{xy}^k}{y(z_{xy})} $$
3.  **Vectorization:**
    Instead of looping pixel-by-pixel, the code processes an entire row $y$ (of width $W$) as a single GPU tensor operation.
4.  **Output:** A tensor `Om_plus` of shape $(30, H, W)$ is saved to disk.

---


In [40]:
# @title Build ω (differentials) for g=30 → Drive (MPS Optimized)
import torch

# -----------------------------
# 1. Initialize or Resume
# -----------------------------
if os.path.exists(OMEGAS_PATH):
    pkg = torch.load(OMEGAS_PATH, map_location='cpu', weights_only=False)
    # Ensure compatibility
    if "meta" in pkg:
        saved_sh = pkg["meta"].get("grid_shape", (0,0))
        if saved_sh != (H, W):
            print(f"Warning: Saved shape {saved_sh} differs from current {(H,W)}.")
    
    Om_plus = pkg["omega_plus"]                  # (g, H, W) complex
    progress = pkg.get("progress", {"iy_done": [0]*genus, "k_done": 0})
    print("Resuming ω from Drive.")
else:
    # Storage uses standard complex64 to save RAM/Disk
    Om_plus = torch.zeros(genus, H, W, dtype=torch.complex64)
    progress = {"iy_done": [0]*genus, "k_done": 0}
    print("Starting fresh ω build.")

# -----------------------------
# 2. MPS Setup for this cell
# -----------------------------
# Manually determine the real component type to avoid attribute errors
if DTYPE == torch.complex128:
    REAL_DTYPE = torch.float64
else:
    REAL_DTYPE = torch.float32

# Move grid_r to GPU once
grid_r_tensor = torch.tensor(grid_r, device=DEVICE, dtype=REAL_DTYPE)

SAVE_EVERY_N_ROWS = 20  # Save less frequently since compute is faster

t0 = time.time()

# -----------------------------
# 3. Main Loop
# -----------------------------
for k in range(progress["k_done"], genus):
    start_iy = int(progress["iy_done"][k])
    if start_iy >= H:
        print(f"ω[{k}] already complete ({H} rows).")
        progress["k_done"] = k+1
        continue

    print(f"Building ω[{k}] rows {start_iy}..{H-1}")
    rows_since_save = 0
    
    # Iterate over rows (Y axis)
    for iy in tqdm(range(start_iy, H), desc=f"omega[{k}]"):
        y_val = grid_i[iy]
        
        # --- VECTORIZED BLOCK START ---
        
        # 1. Create the Complex Row Z = x + iy on MPS
        # shape: (W,)
        Z_row = torch.complex(grid_r_tensor, torch.full_like(grid_r_tensor, y_val))
        
        # 2. Compute Omega using the GPU-optimized function from Cell 1
        # returns shape (W,) on MPS
        row_vals = omega_torch(k, Z_row)
        
        # 3. Move result to CPU and store
        Om_plus[k, iy, :] = row_vals.cpu()
        
        # --- VECTORIZED BLOCK END ---

        progress["iy_done"][k] = iy + 1
        rows_since_save += 1

        # Periodic Save
        if rows_since_save >= SAVE_EVERY_N_ROWS:
            payload = {**COMMON_META, "omega_plus": Om_plus, "progress": progress}
            atomic_torch_save(payload, OMEGAS_PATH)
            rows_since_save = 0

    # Finalize this k
    progress["k_done"] = k+1
    payload = {**COMMON_META, "omega_plus": Om_plus, "progress": progress}
    atomic_torch_save(payload, OMEGAS_PATH)

t1 = time.time()
print(f"ω table saved to {OMEGAS_PATH}  | elapsed {t1 - t0:.1f}s")

Starting fresh ω build.
Building ω[0] rows 0..95


omega[0]: 100%|███████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:00<00:00, 1000.61it/s]


Building ω[1] rows 0..95


omega[1]: 100%|███████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:00<00:00, 1805.05it/s]


Building ω[2] rows 0..95


...

Building ω[28] rows 0..95


omega[28]: 100%|██████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:00<00:00, 1981.55it/s]


Building ω[29] rows 0..95


omega[29]: 100%|██████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:00<00:00, 2031.84it/s]

ω table saved to ./AJ_Tables_g30/aj_omegas_genus30.pt  | elapsed 1.8s



### **Cell 4: Computation of the Abel-Jacobi Map (Integrals)**

**Mathematical Context:**
This cell performs the numerical integration required for the Abel-Jacobi map.

1.  **The Integral:** We compute the period integrals from a fixed base point $P_0$ to every point $z$ on the grid:
    $$ I_k(z) = \int_{P_0}^{z} \omega_k = \int_{P_0}^{z} \frac{t^k}{\sqrt{P(t)}} dt $$

2.  **Path:** The integration path is a straight line segment defined by the parameterization:
    $$ \gamma(t) = P_0 + t(z - P_0), \quad t \in [0, 1] $$

3.  **Numerical Method (Trapezoidal Rule on MPS):**
    The integral is approximated using a discrete summation with $N$ steps (default 500):
    $$ \int_{P_0}^{z} \omega_k \approx \frac{z - P_0}{N} \sum_{j=0}^{N} \omega_k\left( P_0 + \frac{j}{N}(z - P_0) \right) $$
    
    This is executed in parallel for all $z$ in a row and all steps $j$ using matrix operations on the GPU.

4.  **Output:** A tensor `I_plus` of shape $(30, H, W)$, representing the partially computed Abel map values.

___

In [41]:
# @title Build I (integrals) for g=30 → Drive (MPS Optimized)
import torch
import os
import time
from tqdm import tqdm

# -----------------------------
# 0. HOTFIX: Redefine the Integrator to fix 'real_dtype' error
# -----------------------------
def integrate_row_mps(k, z_row_tensor, base_pt, steps=400):
    """
    Computes integral from base_pt to every point in z_row_tensor simultaneously.
    Overwrites the Cell 1 version to fix AttributeError on older PyTorch versions.
    """
    # 1. Determine correct real dtype manually
    if z_row_tensor.dtype == torch.complex128:
        rdtype = torch.float64
    else:
        rdtype = torch.float32

    # 2. Create Time Steps (0 to 1)
    # shape: (steps, 1)
    t = torch.linspace(0, 1, steps, device=z_row_tensor.device, dtype=rdtype).unsqueeze(1)
    
    # 3. Broadcast Path
    diff = z_row_tensor - base_pt
    
    # path shape: (steps, W)
    # We cast t to the complex type of z_row_tensor to ensure smooth broadcasting
    path = base_pt + (t.type(z_row_tensor.dtype) * diff.unsqueeze(0))
    
    # 4. Evaluate Derivative (dt)
    dt = diff / steps
    
    # 5. Evaluate Omega on the whole grid
    vals = omega_torch(k, path)
    
    # 6. Sum (Trapezoidal approximation)
    integral = torch.sum(vals, dim=0) * dt
    return integral

# -----------------------------
# 1. Initialize or Resume
# -----------------------------
if os.path.exists(INTEGRALS_PATH):
    pkg = torch.load(INTEGRALS_PATH, map_location='cpu', weights_only=False)
    ensure_config_compatible(pkg)
    I_plus = pkg["I_plus"]                       # (g, H, W) complex
    progress = pkg.get("progress", {"iy_done": [0]*genus, "k_done": 0})
    print("Resuming integrals from Drive.")
else:
    # Storage on CPU using complex64 to save RAM
    I_plus = torch.zeros(genus, H, W, dtype=torch.complex64)
    progress = {"iy_done": [0]*genus, "k_done": 0}
    print("Starting fresh integrals build.")

# -----------------------------
# 2. MPS Configuration
# -----------------------------
SAVE_EVERY_N_ROWS = 10  # Save less often now that it's faster
STEPS_QUALITY = 500     # Integration steps (Riemann sum).

# Determine real dtype manually for the grid
if DTYPE == torch.complex128:
    REAL_DTYPE = torch.float64
else:
    REAL_DTYPE = torch.float32

# Prepare grid X coordinates on GPU once
grid_r_tensor = torch.tensor(grid_r, device=DEVICE, dtype=REAL_DTYPE)

t0 = time.time()

# -----------------------------
# 3. Main Loop
# -----------------------------
for k in range(progress["k_done"], genus):
    start_iy = int(progress["iy_done"][k])
    if start_iy >= H:
        print(f"I[{k}] already complete ({H} rows).")
        progress["k_done"] = k+1
        continue

    print(f"Building I[{k}] rows {start_iy}..{H-1}")
    rows_since_save = 0
    
    # Iterate over rows (Y axis)
    for iy in tqdm(range(start_iy, H), desc=f"I[{k}]   "):
        y_val = grid_i[iy]
        
        # --- VECTORIZED BLOCK START ---
        
        # 1. Create the Complex Row Z = x + iy on MPS
        # shape: (W,)
        Z_row = torch.complex(grid_r_tensor, torch.full_like(grid_r_tensor, y_val))
        
        # 2. Compute Integral for the whole row
        # Now uses the patched integrate_row_mps defined above
        row_vals = integrate_row_mps(k, Z_row, base_point_complex, steps=STEPS_QUALITY)
        
        # 3. Move result to CPU and store
        I_plus[k, iy, :] = row_vals.cpu()
        
        # --- VECTORIZED BLOCK END ---

        progress["iy_done"][k] = iy + 1
        rows_since_save += 1

        # Periodic Save
        if rows_since_save >= SAVE_EVERY_N_ROWS:
            payload = {**COMMON_META, "I_plus": I_plus, "progress": progress}
            atomic_torch_save(payload, INTEGRALS_PATH)
            rows_since_save = 0

    # Finalize this k
    progress["k_done"] = k+1
    payload = {**COMMON_META, "I_plus": I_plus, "progress": progress}
    atomic_torch_save(payload, INTEGRALS_PATH)

t1 = time.time()
print(f"I table saved to {INTEGRALS_PATH}  | elapsed {t1 - t0:.1f}s")

Starting fresh integrals build.
Building I[0] rows 0..95


I[0]   : 100%|█████████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:00<00:00, 354.63it/s]


Building I[1] rows 0..95


I[1]   : 100%|█████████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:00<00:00, 628.51it/s]


Building I[2] rows 0..95


...

Building I[28] rows 0..95


I[28]   : 100%|████████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:00<00:00, 632.20it/s]


Building I[29] rows 0..95


I[29]   : 100%|████████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:00<00:00, 595.82it/s]

I table saved to ./AJ_Tables_g30/aj_integrals_genus30.pt  | elapsed 4.5s
